In [1]:
import pyspark.pandas as ps
from pyspark import SparkContext
from pyspark.sql import SparkSession
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

warnings.filterwarnings('ignore')

spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .config("spark.ui.showConsoleProgress", "false") \
    .config("spark.jars", """
            /usr/local/spark/jars/aws-java-sdk-bundle-1.12.262.jar,
            /usr/local/spark/jars/antlr4-runtime-4.9.3.jar,
            /usr/local/spark/jars/delta-contribs_2.12-3.1.0.jar,
            /usr/local/spark/jars/delta-iceberg_2.12-3.1.0.jar,
            /usr/local/spark/jars/delta-spark_2.12-3.1.0.jar,
            /usr/local/spark/jars/delta-storage-3.1.0.jar,
            /usr/local/spark/jars/hadoop-aws-3.3.4.jar""") \
    .appName("SparkJupyter") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

%load_ext sparksql_magic
%config SparkSql.limit=20

/opt/conda/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(
24/08/22 04:08:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
spark.sql("CREATE DATABASE IF NOT EXISTS BIdatabase")
spark.sql("USE BIdatabase")

DataFrame[]

In [3]:
df = spark.read.csv("s3a://data/new_retail_data.csv", header=True, inferSchema=True)

In [4]:
from pyspark.sql.functions import col, mean, sum

df = spark.read.csv("s3a://data/new_retail_data.csv", header=True, inferSchema=True)
df = df.withColumn("Total_Purchases", col("Total_Purchases").cast("int"))
df = df.withColumn("Age", col("Age").cast("int"))
df = df.withColumn("Year", col("Year").cast("string"))
df = df.withColumn("Month", col("Month").cast("string"))
df = df.withColumn("Transaction_ID", col("Transaction_ID").cast("int"))
df = df.withColumn("Customer_ID", col("Customer_ID").cast("int"))
df = df.dropna()
df = df.dropDuplicates()
df.write.mode("overwrite").saveAsTable("BIdatabase.general_table")

In [5]:
dfm = spark.read.parquet("s3a://data/df_rfm", header=True, inferSchema=True)
dfm = dfm.dropna()
dfm = dfm.dropDuplicates()
dfm.write.mode("overwrite").saveAsTable("BIdatabase.rfm_table")

In [6]:
spark.sql("DESCRIBE database BIdatabase").show(200)

+--------------+--------------------+
|     info_name|          info_value|
+--------------+--------------------+
|  Catalog Name|       spark_catalog|
|Namespace Name|          BIdatabase|
|       Comment|                    |
|      Location|s3a://data/wareho...|
|         Owner|              jovyan|
+--------------+--------------------+

